<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [524]:
import pandas as pd
import psycopg2

In [525]:
'''
Задаем конфиг для коннекта к БД
'''
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [526]:
# Коннектимся к БД
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [527]:
# текст запроса
query_3_1 = f'''select count(id) from vacancies
'''

In [528]:
# вывод результата запроса
answer_3_1 = pd.read_sql_query(query_3_1, connection)
display(answer_3_1)

/tmp/ipykernel_458/2616261428.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_3_1 = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [529]:
# текст запроса
query_3_2 = f'''
select count(id) from employers
'''

In [530]:
# вывод результата запроса
answer_3_2 = pd.read_sql_query(query_3_2, connection)
display(answer_3_2)

/tmp/ipykernel_458/1450756444.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_3_2 = pd.read_sql_query(query_3_2, connection)


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [531]:
# текст запроса
query_3_3 = f'''
select count(id) from areas
'''

In [532]:
# вывод результата запроса
answer_3_3 = pd.read_sql_query(query_3_3, connection)
display(answer_3_3)

/tmp/ipykernel_458/870207647.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_3_3 = pd.read_sql_query(query_3_3, connection)


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [533]:
#текст запроса
query_3_4 = f'''
select count(id) from industries
'''

In [534]:
#вывод результата запроса
answer_3_4 = pd.read_sql_query(query_3_4, connection)
display(answer_3_4)

/tmp/ipykernel_458/2108994857.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_3_4 = pd.read_sql_query(query_3_4, connection)


,count
0,294


## Выводы
- На 1 работадателя в средмем приходится, примерно, по 2 позиции
- Количество регионов превосходит количество регионов в России в их классическом понимании, значит, или имеются некорректные данные в справочнике (которые впоследствии можно нормализировать) и/или вакансии также находятся заграницей
- Большое количество индустрий в справочной таблице industries, являются потенциальным кандидатом для нормализации
***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [535]:
#текст запроса
query_4_1 = f'''
select
    a.name,
    count (v.id)
from
    vacancies as v
    join areas as a on a.id = v.area_id
group by a.id -- группируем по id, не по имени, т.к имя может совпадать
order by 2 desc -- сортируем по количеству
limit 5
'''

In [536]:
#вывод результата запроса
answer_4_1 = pd.read_sql_query(query_4_1, connection)
display(answer_4_1)

/tmp/ipykernel_458/2180171517.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_4_1 = pd.read_sql_query(query_4_1, connection)


,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [537]:
#текст запроса
query_4_2 = f'''
select count (id)
from vacancies
where
    salary_from is not null
    or salary_to is not null
'''

In [538]:
#вывод результата запроса
answer_4_2 = pd.read_sql_query(query_4_2, connection)
display(answer_4_2)

/tmp/ipykernel_458/2052512690.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_4_2 = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [539]:
#текст запроса
query_4_3 = f'''
select
    round(avg(salary_from)) as mean_from, -- округляем среднее значение поля salary_from
    round(avg(salary_to)) as mean_to -- округляем среднее значение поля salary_from
from vacancies
'''

In [540]:
#вывод результата запроса
answer_4_3 = pd.read_sql_query(query_4_3,connection)
display(answer_4_3)

/tmp/ipykernel_458/2972828627.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_4_3 = pd.read_sql_query(query_4_3,connection)


,mean_from,mean_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [541]:
# текст запроса
query_4_4 = f'''
select
    distinct (schedule,employment), -- выбираем уникальные сочетания полей schedule и employement
    count(id) -- считаем количство подходящих вакансий
from vacancies
group by 1 -- группируем по сочетанию полей schedule и employement
order by 2 desc -- сортируем по количеству
'''

In [542]:
#вывод результата запроса
answer_4_4 = pd.read_sql_query(query_4_4,connection)
display(answer_4_4)

/tmp/ipykernel_458/1475180542.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_4_4 = pd.read_sql_query(query_4_4,connection)


,row,count
0,"(""Полный день"",""Полная занятость"")",35367
1,"(""Удаленная работа"",""Полная занятость"")",7802
2,"(""Гибкий график"",""Полная занятость"")",1593
3,"(""Удаленная работа"",""Частичная занятость"")",1312
4,"(""Сменный график"",""Полная занятость"")",940
5,"(""Полный день"",Стажировка)",569
6,"(""Вахтовый метод"",""Полная занятость"")",367
7,"(""Полный день"",""Частичная занятость"")",347
8,"(""Гибкий график"",""Частичная занятость"")",312
9,"(""Полный день"",""Проектная работа"")",141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [543]:
# текст запроса
query_4_5 = f'''
select experience
from vacancies
group by experience -- группируем по опыту работы
order by count(id) asc -- сортируем в порядке возрастания количества вакансий
'''

In [544]:
#вывод результата запроса
answer_4_5 = pd.read_sql_query(query_4_5, connection)
display(answer_4_5)

/tmp/ipykernel_458/2279721042.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_4_5 = pd.read_sql_query(query_4_5, connection)


,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


## Выводы
- Больше половина вакансий не содержит укзания зарплатной вилки, это может говорить о
  - Работадатели предпочитают обсуждать зар. плату индивидуально с работками
  - Вероятно выгоднее получать больше откликов на вакансию и производить отсев до этого, чем лишаться части откликов
- На рынке в самом большом спросе находятся ресурсы с опытом от 1 до 3 лет, в наименьшем люди с солидным опытом, что можно интерпретировать как запрос рынка на middle / middle + &senior / junior / senior +
- Небольшое количество стажировок в % соотношение, может являться косвенным свидельством умеренного спроса на junior ресурсы
- Наиболее требуемой формой рабочего контракта является полный день и полная занятость, что в пост ковидный период выглядит странным, возможно представленные данные
- Больше всего вакансий находится в Москве, однако в разрезе количества населения количество предложений выглядит скромнее чем в Новосибирске или СПБ, возможно, это всязано с высокой конкуренцией в Москве
***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [545]:
# тест запроса
query_5_1 = f'''
(select e.name
from
    employers as e
    join vacancies as v on e.id = v.employer_id
group by e.id -- группируем по id, не по имени, чтобы убедиться в
order by count(v.id) desc -- производим сортировку объединяемых таблиц отдельно
limit 1) -- выбираем первую позицию

union all -- объединяем первую и пятую позиции

(select e.name
from
    employers as e
    join vacancies as v on e.id = v.employer_id
group by e.name
order by count(v.id) desc
offset 4 -- выбираем 5ую позицию
limit 1)
'''

In [546]:
#вывод результата запроса
answer_5_1 = pd.read_sql_query(query_5_1, connection)
display(answer_5_1)

/tmp/ipykernel_458/2844143951.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5_1 = pd.read_sql_query(query_5_1, connection)


,name
0,Яндекс
1,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [547]:
# текст запроса
query_5_2 = f'''
select
    a.name,
    count(distinct e.id) as cnt_employers, -- считаем количество работадателей
    count(v.id) as cnt_vacancies -- считаем количество вакансий
from
    areas a
    left join employers e on e.area = a.id
    left join vacancies v on
        v.employer_id = e.id
        and v.area_id = a.id
group by a.id
having count(v.id) = 0 --выбираем только те регионы где нет ни 1 вакансии
order by 2 desc -- сортируем по количеству работодателей
limit 1 --выбриаем регион с наибольшим количеством работодателей
'''

In [548]:
#вывод результата запроса
answer_5_2 = pd.read_sql_query(query_5_2, connection)
display(answer_5_2)

/tmp/ipykernel_458/3352381363.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5_2 = pd.read_sql_query(query_5_2, connection)


,name,cnt_employers,cnt_vacancies
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [549]:
# текст запроса
query_5_3 = f'''
select
    e.name,
    count(distinct v.area_id)
from
    employers as e
    left join vacancies as v on e.id = v.employer_id
group by e.id
order by 2 desc
'''

In [550]:
#вывод результата запроса
answer_5_3 = pd.read_sql_query(query_5_3, connection)
display(answer_5_3)

/tmp/ipykernel_458/3661101528.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5_3 = pd.read_sql_query(query_5_3, connection)


,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [551]:
# текст запроса
query_5_4 = f'''
select
    count(e.id)
from
    employers as e
    left join employers_industries as ei on ei.employer_id = e.id
where ei.employer_id is null
'''

In [552]:
#вывод результата запроса
answer_5_4 = pd.read_sql_query(query_5_4, connection)
display(answer_5_4)

/tmp/ipykernel_458/2254713031.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5_4 = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [553]:
# текст запроса
query_5_5 = f'''
select
    e.name
from
    employers as e
    join employers_industries as ei on ei.employer_id = e.id
group by e.id
having count(ei.employer_id) = 4
order by e.name
offset 2
limit 1
'''

In [554]:
#вывод результата запроса
answer_5_5 = pd.read_sql_query(query_5_5, connection)
display(answer_5_5)

/tmp/ipykernel_458/2620705473.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5_5 = pd.read_sql_query(query_5_5, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [555]:
# текст запроса
query_5_6 = f'''
select
    count(e.id)
from
    employers as e
    join employers_industries as ei on ei.employer_id = e.id
    join industries as i on ei.industry_id = i.id
where i.name = 'Разработка программного обеспечения'
'''

In [556]:
#вывод результата запроса
answer_5_6 = pd.read_sql_query(query_5_6, connection)
display(answer_5_6)

/tmp/ipykernel_458/3430973104.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5_6 = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [557]:
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url) # Выполняем get запрос
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер

table = page.find_all('table', class_ = 'standard')[0] # Из всех таблиц выбираем первую, встречающуюся
links = table.find_all('a') # ищем все ссылке в выбранной таблице

list_cities = [link.text for link in links[4:]] # при помощи генератора созаем список городов, отсекая сслыки, не имеющие смысла
list_cities.remove('[a]') # удаляем еще 1 лишнюю ссылку
list_cities = tuple(list_cities) # преобразуем в кортеж, который будет удобно использовать в запросе
display(list_cities)


('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [558]:
# текст запроса
query_5_7 = f'''
(select
    a.name,
    count(v.id)
from
    employers e
    join vacancies v on v.employer_id = e.id and e.name = 'Яндекс' -- производим join только тех записей, где работодатель Яндрекс
    join areas a on a.id = v.area_id
where
    a.name in {list_cities} -- проверяем условие вхождение региона в сприсок list_cities
group by a.id -- группируем по id региона
order by 2 desc) -- сортируем по количеству в порядке убываения

union all -- объедияем с Total

select
    'total',
    count(v.id)
from
    employers e
    join vacancies v on v.employer_id = e.id and e.name = 'Яндекс'
    join areas a on a.id = v.area_id
where
    a.name in {list_cities}
'''

In [559]:
#вывод результата запроса
answer_5_7 = pd.read_sql_query(query_5_7,connection)
display(answer_5_7)

/tmp/ipykernel_458/2069820079.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5_7 = pd.read_sql_query(query_5_7,connection)


,name,count
0,total,485
1,Москва,54
2,Санкт-Петербург,42
3,Екатеринбург,39
4,Нижний Новгород,36
5,Новосибирск,35
6,Воронеж,32
7,Краснодар,30
8,Самара,26
9,Уфа,26


***

## Выводы
- На примере Яндекса можно косвенно подтвердить вывод, о том, что нет тотальной доминации Москва по количетсву вакансий, регионы в портфеле Яндекса, скорее соответствуют количеству университетов по городам России
- Среди наиболее крупных работодателей доминируют часные фирмы, это может быть следствием их более динамичного поиска сотрудников
- Значительное количество работодателей без привязки к индустрии может говорить о том, что эти работодатели не запостили объявлений или их доля в отрытых вакансиях митимальна
- Самая популярная сфера деятельности - разработка програмного обеспечения, и области ассоциированные с ИТ вероятно, она генерирет значительное количе
- Имеется определенная ценрализация работодателей в МСК и СПБ, они имеют больше работодателей, чем вакансий, другие регионы имеют больше вакансий, чем работодателей, что иллюстрирует тендецию работы на нелокального работодателя

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [560]:
# текст запроса
query_6_1 =f'''
select
    count(id)
from vacancies
where
    name ~*'data' --используем regexp чтобы выбрать вакансии содержащие 'data', нечувст к регистру
    or name ~*'данн' --делаем тоже самое для строки 'данн'
'''

In [561]:
#вывод результата запроса
answer_6_1 = pd.read_sql_query(query_6_1, connection)
display(answer_6_1)

/tmp/ipykernel_458/691852798.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_6_1 = pd.read_sql_query(query_6_1, connection)


,count
0,1771



2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [562]:
# текст запроса

# создаем базу для последующих запросов, вакансии, имеющие отношения к DS
query_6 = f'''
select *
from vacancies
where
    (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or name like '%ML%'
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and name not like '%HTML%'
'''

# создаем запрос на выборку всех джуниорских позиций
condition_6_2 = f'''
select *
from vacancies
where
    name ilike '%junior%'
    or experience = 'Нет опыта'
    or employment = 'Стажировка'
'''

# создаем завпрос на как пересечение вакансий DS и джуниорских
query_6_2 = f'''
select count(*)
from (
    {query_6}
intersect
    {condition_6_2}
) x
'''

In [563]:
#вывод результата запроса

answer_6_2 = pd.read_sql_query(query_6_2, connection)
display(answer_6_2)

/tmp/ipykernel_458/2911736217.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_6_2 = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [564]:
# текст запроса

# делаем запрос на выборку вакансий в которых указан SQL или postgres
condition_6_3 = f'''
select *
from vacancies
where
    key_skills ilike '%SQL%'
    or key_skills ilike '%postgres%'
'''
# создаем запрос как пересечение результатов запроса на вакансии DS и те, в которых указаны SQL или postgres
query_6_3 = f'''
select count(*)
from (
    {query_6}
intersect
    {condition_6_3}
) x
'''

In [565]:
#вывод результата запроса
answer_6_3 = pd.read_sql_query(query_6_3, connection)
display(answer_6_3)

/tmp/ipykernel_458/1670678677.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_6_3 = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [566]:
# текст запроса

# делаем запрос на выборку вакансий в которых указан Python
condition_6_4 = f'''
select *
from vacancies
where
    key_skills ilike '%python%'
'''
# создаем запрос как пересечение результатов запроса на вакансии DS и те, в которых указан Python
query_6_4 = f'''
select count(*)
from (
    {query_6}
intersect
    {condition_6_4}
) x
'''


In [567]:
#вывод результата запроса

answer_6_4 = pd.read_sql_query(query_6_4, connection)
display(answer_6_4)

/tmp/ipykernel_458/229229148.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_6_4 = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [568]:
# текст запроса
'''
Для выполнения этого задания исходим из предположения, что скиллы разделены друг от друга символом табуляции
Подсчет количества скиллов осуществляется вычислением длины изначальной строки key_skills и строки с удаленными символами табуляции
'''

query_6_5 = f'''
select
    AVG(length (key_skills) - length (replace(key_skills, chr(9),''))+1)
from ({query_6}) x
'''

In [569]:
#вывод результата запроса

answer_6_5 = pd.read_sql_query(query_6_5,connection)
display(answer_6_5)

/tmp/ipykernel_458/3806489783.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_6_5 = pd.read_sql_query(query_6_5,connection)


,avg
0,6.406032


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [570]:
# текст запроса

query_6_6 = f'''
select
    experience,
    avg(
        case when -- задаем условие того какое значение брать в зависимости от комбинации наличия полей salary_from и salary_to
            salary_from is not null
            and salary_to is not null
        then
            (salary_from + salary_to) / 2
        when
            salary_from is null
        then
            salary_to
        else -- т.е если salary_to is null
            salary_from
        end
)
from (
    {query_6} -- из вакансий, имеющих отношение к DS

intersect -- исключаем дополнительно вакансии у которых не указана заралатная вилка

select *
from vacancies
where
    salary_from is not null
    or salary_to is not null) x

group by experience -- группируем результаты по опыту работы
'''

In [571]:
#вывод результата запроса

answer_6_6 = pd.read_sql_query(query_6_6, connection)
display(answer_6_6)

/tmp/ipykernel_458/2528280730.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_6_6 = pd.read_sql_query(query_6_6, connection)


,experience,avg
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


***

## Выводы
- Вакансии DS составляют ~ 1% от общего числа, что говорит о нишевости работы или о ее низкой востребованности
- Выборка вакансий, имеющих отношение к DS не оптимальна, т.к в результаты могут попасть такие технологии как QML, не относящиеся к DS
- Зарлатная вилка у вакансий DS соотвествует средним значениям по нижней границе, значительно превосходит средние значения по верхней, посчитанным в части 2
- Отсуствует зарплатная вилка у DS вакансий с опытом работы 6+, что может быть объясняться высокими зараплатными ожиданиями
- SQL и Postgre требуются примерно в 41% DS вакансий
- Python в 73%
***

# Общий вывод по проекту
- Изначальные данные, вероятнего всего нуждаются в дополнительной обработке
- Москва продолжает оставаться центром развития с самым большим количеством работадателей и доступных вакансий, однако, в разрезе населения лидерство
- Большинство работодателей находятся в МСК и СПБ, одноко, наимают больше людей из регионов
- Общие тенденции характерны для DS вакансий в целом (см запросы 7.1, 7.2)
- Примерно 25% от всех вакансий были предложены работодателями, занимающимися разработкой PO (см 7.3)
- DS представлены в 97 различных областях (см 7.4)

In [572]:
'''
Смотрим на среднее количество скилов, указаных в вакансиях

Для выполнения этого задания исходим из предположения, что скиллы разделены друг от друга символом табуляции
Подсчет количества скиллов осуществляется вычислением длины изначальной строки key_skills и строки с удаленными символами табуляции
'''

query_7_1 = f'''
select
    AVG(length (key_skills) - length (replace(key_skills, chr(9),''))+1)
from vacancies
'''

answer_7_1 = pd.read_sql_query(query_7_1, connection)
display(answer_7_1)


/tmp/ipykernel_458/3516376034.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_7_1 = pd.read_sql_query(query_7_1, connection)


,avg
0,6.370339


In [573]:
'''
Смотрим на количество DS вакансий в разрезе опыта
'''
query_7_2 = f'''
select
    experience,
    count (id)
from vacancies
where
    (name ilike '%data scientist%'
    or name ilike '%data science%'
    or name ilike '%исследователь данных%'
    or name like '%ML%'
    or name ilike '%machine learning%'
    or name ilike '%машинн%обучен%')
    and name not like '%HTML%'
group by experience
'''

answer_7_2 = pd.read_sql_query(query_7_2, connection)
display(answer_7_2)

/tmp/ipykernel_458/3213259416.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_7_2 = pd.read_sql_query(query_7_2, connection)


,experience,count
0,Более 6 лет,34
1,Нет опыта,28
2,От 1 года до 3 лет,223
3,От 3 до 6 лет,195


In [574]:
'''
Смотрим сколько вакансий опубликовали компании со сферой деятельности Разработка ПО
'''

query_7_3 = f'''
select
    e.name,
    count(v.id)
from
    employers as e
    join employers_industries as ei on ei.employer_id = e.id
    join industries as i on ei.industry_id = i.id
    join vacancies as v on v.employer_id = e.id
where i.name = 'Разработка программного обеспечения'
group by e.id

union all

select
    'total',
    count(v.id)
from
    employers as e
    join employers_industries as ei on ei.employer_id = e.id
    join industries as i on ei.industry_id = i.id
    join vacancies as v on v.employer_id = e.id
where
    i.name = 'Разработка программного обеспечения'

order by 2 desc
'''

answer_7_3 = pd.read_sql_query(query_7_3, connection)
display(answer_7_3)

/tmp/ipykernel_458/1914913161.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_7_3 = pd.read_sql_query(query_7_3, connection)


,name,count
0,total,12499
1,Тинькофф,444
2,DataArt,247
3,Совкомбанк Технологии,204
4,IBS,175
...,...,...
2588,ГК ГЕЛИОС,1
2589,Meanotek,1
2590,Competent Solutions,1
2591,Илар Групп,1


In [575]:
'''
Смотрим в скольких различных сферах требуются DS специалисты
'''

query_7_4 = f'''
select count(distinct i.id)
from
    vacancies v
    join employers e on e.id = v.employer_id
    join employers_industries ei on ei.employer_id = e.id
    join industries i on i.id = ei.industry_id
where
    (v.name ilike '%data scientist%'
    or v.name ilike '%data science%'
    or v.name ilike '%исследователь данных%'
    or v.name like '%ML%'
    or v.name ilike '%machine learning%'
    or v.name ilike '%машинн%обучен%')
    and v.name not like '%HTML%'
'''

answer_7_4 = pd.read_sql_query(query_7_4, connection)
display(answer_7_4)

/tmp/ipykernel_458/3174420425.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_7_4 = pd.read_sql_query(query_7_4, connection)


,count
0,97


In [576]:
connection.close() # закрываем connection